# Machine Translation Project

## Introduction
In this notebook, I have built a deep neural network that functions as part of an end-to-end machine translation pipeline. The pipeline accepts English text as input and returns the French translation. There are three main steps:

- **Preprocess** - Convert text to sequence of integers.
- **Models** - Create models which accept a sequence of integers as input and return a probability distribution over possible translations. I have experimented with 5 different models and the final one, called `model_final`, is the best performer, achieving an accuracy of 99%.
- **Prediction** - Run the best model on English text.

## Table of contents
1) [Dataset](#dataset)

2) [Preprocess](#preprocess)

3) [Models](#models)

4) [Prediction](#prediction)

## 1) Dataset <a class="anchor" id="dataset"></a>
First, let's investigate the dataset that will be used to train and evaluate my pipeline. The most common datasets used for machine translation are from [WMT](http://www.statmt.org/). However, that will take a long time to train a neural network on. I'll be using a dataset Udacity created for this project that contains a small vocabulary. I'll be able to train my model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [1]:
import helper
from keras.layers import GRU, LSTM, Input, Dense, TimeDistributed, Bidirectional, Dropout, SpatialDropout1D
from keras.models import Model
from keras.layers import Activation
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.layers import RepeatVector
from keras.layers.embeddings import Embedding

# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Using TensorFlow backend.


Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [2]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, we can see they have been preprocessed already. The punctuations have been delimited using spaces. All the text has been converted to lowercase. However, the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary. A more complex vocabulary is a more complex problem. Let's look at the complexity of the dataset.

In [3]:
import collections

english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## 2) Preprocess <a class="anchor" id="preprocess"></a>
For this project, I'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

### Tokenize
For a neural network to predict on text data, the data first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number. These are called character and word ids, respectively. Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word. Word level models tend to learn better, since they are lower in complexity, so I'll use those.

In the following cell, `tokenize` turns each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Running the cell will run `tokenize` on sample data and show output for debugging.

In [4]:
import project_tests as tests
from keras.preprocessing.text import Tokenizer

def tokenize(x):
    """Tokenize x.
    
    Parameters:
    x: List of sentences/strings to be tokenized
    
    Returns: 
    Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    sequences = tokenizer.texts_to_sequences(x)
    return sequences, tokenizer

tests.test_tokenize(tokenize)

# Tokenize example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding
When batching the sequence of word ids together, each sequence needs to be the same length. Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [5]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences

def pad(x, length=None):
    """Pad x.
    Parameters:
    x: List of sequences
    length: Length to pad the sequence to. If None, use length of longest sequence in x
    
    Returns: 
    Padded numpy array of sequences
    """
    if length == None:
        longest_sequence = max(x, key=len)
        return pad_sequences(x, len(longest_sequence), padding='post')
    return pad_sequences(x, length, padding='post')

tests.test_pad(pad)

# Pad tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
The `preprocess` function tokenizes and pads feature and label sentences.

In [6]:
def preprocess(x, y):
    """Preprocess x and y.
    
    Parameters:
    x: Feature list of sentences
    y: Label list of sentences
    
    Returns: 
    Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)

print('Data Preprocessed')

Data Preprocessed


In [7]:
print (preproc_english_sentences.shape)
print (preproc_french_sentences.shape)

(137861, 15)
(137861, 21, 1)


## 3) Models <a class="anchor" id="models"></a>
In this section, I will experiment with various neural network architectures.
I will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an Encoder-Decoder RNN

After experimenting with the four simple architectures, I will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want. We want the French translation. The function `logits_to_text` will bridge the gap between the logits from the neural network to the French translation.

In [8]:
def logits_to_text(logits, tokenizer):
    """Turn logits from a neural network into text using the tokenizer.
    
    Parameters:
    logits: Logits from a neural network
    tokenizer: Keras Tokenizer fit on the labels
    
    Returns: 
    String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    print ("logist shape: ", logits.shape)

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, I'll build a RNN that translates English to French.

In [9]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """Build and train a basic RNN on x and y.
    Parameters:
    input_shape: Tuple of input shape
    output_sequence_length: Length of output sequence
    english_vocab_size: Number of unique English words in the dataset
    french_vocab_size: Number of unique French words in the dataset
    
    Returns: 
    Keras model built, but not trained
    """
    learning_rate = 0.01
    inputs = Input(shape=input_shape[1:])
    x = GRU(512, return_sequences=True)(inputs)
    x = TimeDistributed(Dense(french_vocab_size, activation='relu'))(x)
    predictions = Activation('softmax')(x)
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)


# Reshape the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index) + 1, # Add 1 because padding introduces 0
    len(french_tokenizer.word_index) + 1) # Add 1 because padding introduces 0
simple_rnn_model.summary()
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 21, 1)             0         
_________________________________________________________________
gru_2 (GRU)                  (None, 21, 512)           789504    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 21, 345)           176985    
_________________________________________________________________
activation_2 (Activation)    (None, 21, 345)           0         
Total params: 966,489
Trainable params: 966,489
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 sample

In [10]:
simple_rnn_model_scores = simple_rnn_model.evaluate(tmp_x, preproc_french_sentences, verbose=0)
print("Model Accuracy: %.2f%%" % (simple_rnn_model_scores[1]*100))

Model Accuracy: 67.84%


### Model 2: Embedding
![RNN](images/embedding.png)
I've turned the words into ids, but there's a better representation of a word. This is called word embeddings. An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, I'll create a RNN model using embedding.

In [11]:
from keras.layers.embeddings import Embedding

def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """Build and train a RNN model using word embedding on x and y.
    
    Parameters:
    input_shape: Tuple of input shape
    output_sequence_length: Length of output sequence
    english_vocab_size: Number of unique English words in the dataset
    french_vocab_size: Number of unique French words in the dataset
    
    Returns: 
    Keras model built, but not trained
    """
    learning_rate = 0.01
    inputs = Input(shape=input_shape[1:])
    x = Embedding(english_vocab_size, output_sequence_length)(inputs)
    x = GRU(512, return_sequences=True)(x)
    x = TimeDistributed(Dense(french_vocab_size, activation='relu'))(x)
    predictions = Activation('softmax')(x)
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_embed_model(embed_model)


# Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])

# Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
print (embed_rnn_model.summary())
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 21)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 21, 21)            4200      
_________________________________________________________________
gru_4 (GRU)                  (None, 21, 512)           820224    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 21, 345)           176985    
_________________________________________________________________
activation_4 (Activation)    (None, 21, 345)           0         
Total params: 1,001,409
Trainable params: 1,001,409
Non-trainable params: 0
_________________________________________________________________
None
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 123s 1ms/step - 

In [12]:
embed_rnn_model_scores = embed_rnn_model.evaluate(tmp_x, preproc_french_sentences, verbose=0)
print("Model Accuracy: %.2f%%" % (embed_rnn_model_scores[1]*100))

Model Accuracy: 90.42%


### Model 3: Bidirectional RNNs
![RNN](images/bidirectional.png)
One restriction of an RNN is that it can't see the future input, only the past. This is where bidirectional recurrent neural networks come in. They are able to see the future data.

In [13]:
from keras.layers import Bidirectional

def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """Build and train a bidirectional RNN model on x and y.
    
    Parameters:
    input_shape: Tuple of input shape
    output_sequence_length: Length of output sequence
    english_vocab_size: Number of unique English words in the dataset
    french_vocab_size: Number of unique French words in the dataset
    
    Returns: 
    Keras model built, but not trained
    """
    learning_rate = 0.01
    inputs = Input(shape=input_shape[1:])
    x = Bidirectional(GRU(512, return_sequences=True))(inputs)
    x = TimeDistributed(Dense(french_vocab_size, activation='relu'))(x)
    predictions = Activation('softmax')(x)
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_bd_model(bd_model)

# Reshape the input to work with model
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
bd_rnn_model = bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
bd_rnn_model.summary()
bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 21, 1)             0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 21, 1024)          1579008   
_________________________________________________________________
time_distributed_6 (TimeDist (None, 21, 345)           353625    
_________________________________________________________________
activation_6 (Activation)    (None, 21, 345)           0         
Total params: 1,932,633
Trainable params: 1,932,633
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 176s 2ms/step - loss: 2.2167 - acc: 0.5684 - val_loss: 1.7328 - val_acc: 0.6239

In [14]:
bd_rnn_model_scores = bd_rnn_model.evaluate(tmp_x, preproc_french_sentences, verbose=0)
print("Model Accuracy: %.2f%%" % (bd_rnn_model_scores[1]*100))

Model Accuracy: 72.68%


### Model 4: Encoder-Decoder
Time to look at encoder-decoder models. This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence. The decoder takes this matrix as input and predicts the translation as output.

In [15]:
from keras.layers import RepeatVector

def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """Build and train an encoder-decoder model on x and y.
    
    Parameters:
    input_shape: Tuple of input shape
    output_sequence_length: Length of output sequence
    english_vocab_size: Number of unique English words in the dataset
    french_vocab_size: Number of unique French words in the dataset
    
    Returns: 
    Keras model built, but not trained
    """
    learning_rate = 0.001
    inputs = Input(shape=input_shape[1:])
    encoded = GRU(512)(inputs)
    
    decoded = RepeatVector(output_sequence_length)(encoded)
    decoded = GRU(256, return_sequences=True)(decoded)
    decoded = TimeDistributed(Dense(french_vocab_size, activation='relu'))(decoded)
    predictions = Activation('softmax')(decoded)
    
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_encdec_model(encdec_model)


# Reshape the input to work with model
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
encdec_rnn_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)
encdec_rnn_model.summary()
encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 21, 1)             0         
_________________________________________________________________
gru_9 (GRU)                  (None, 512)               789504    
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 21, 512)           0         
_________________________________________________________________
gru_10 (GRU)                 (None, 21, 256)           590592    
_________________________________________________________________
time_distributed_8 (TimeDist (None, 21, 345)           88665     
_________________________________________________________________
activation_8 (Activation)    (None, 21, 345)           0         
Total params: 1,468,761
Trainable params: 1,468,761
Non-trainable params: 0
_________________________________________________________________


In [16]:
encdec_rnn_model_scores = encdec_rnn_model.evaluate(tmp_x, preproc_french_sentences, verbose=0)
print("Model Accuracy: %.2f%%" % (encdec_rnn_model_scores[1]*100))

Model Accuracy: 60.16%


### Model 5: Custom
Now I'll create a model that incorporates embedding, encoder-decoder and bidirectional RNN. The code is then followed by a rationale for choosing values for hyperparameters to improve the model's accuracy.

In [11]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y.
    
    Parameters:
    input_shape: Tuple of input shape
    output_sequence_length: Length of output sequence
    english_vocab_size: Number of unique English words in the dataset
    french_vocab_size: Number of unique French words in the dataset
    
    Returns: 
    Keras model built, but not trained
    """
    learning_rate = 0.001
    inputs = Input(shape=input_shape[1:])
    encoded = Embedding(english_vocab_size, 300)(inputs)
    encoded = Bidirectional(LSTM(512, dropout=0.2))(encoded)
    encoded = Dense(512, activation='relu')(encoded)
    
    decoded = RepeatVector(output_sequence_length)(encoded)
    decoded = Bidirectional(LSTM(512, dropout=0.2, return_sequences=True))(decoded)
    decoded = TimeDistributed(Dense(french_vocab_size))(decoded)
    
    predictions = Activation('softmax')(decoded)
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_model_final(model_final)


print('Final Model Loaded')

Final Model Loaded


**Steps taken to tune hyperpameters to improve the final model's accuracy**

Below is the inital attempt that yielded an accuracy of 0.9696:
```
    learning_rate = 0.01
    inputs = Input(shape=input_shape[1:]).
    encoded = Embedding(english_vocab_size, output_sequence_length)(inputs)
    encoded = Bidirectional(GRU(128))(encoded)
    encoded = Dense(128, activation='relu')(encoded)
    
    decoded = RepeatVector(output_sequence_length)(encoded)
    decoded = Bidirectional(GRU(128, return_sequences=True))(decoded)
    decoded = TimeDistributed(Dense(french_vocab_size))(decoded)
    
    predictions = Activation('softmax')(decoded)
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
```
I tuned the following hyperparameters one by one to see the effect. Below were the final values chosen: 
* Decreased the `learning_rate` from 0.01 to 0.001
* Increased the output dimension, `output_dim`, of the Embedding layer from 15 to 300
* Used `LSTM` instead of `GRU` in `Bidirectional`. I had initially picked GRUs because the dataset was small and GRUs, according to [this source](https://datascience.stackexchange.com/questions/14581/when-to-use-gru-over-lstm), train faster and perform better than LSTMs on less training data. However, it turned out that LSTMs did better. As noted in the same source, LSTMs should in theory remember longer sequences than GRUs and outperform them in tasks requiring modeling long-distance relations.
* Increased the hidden units from 128 to 512. This gave the model more learning capacity.
* Used `dropout` in the hidden layers LSTM. As the number of hidden units increased, I added `dropout` to reduce overfitting.
* Increased the number of epochs from 20 to 30. The validation error decreased as the number of epochs increased. However, it didn't fall any further from epoch 30 onwards.

The final model, as shown above in the `model_final` function, achieves an accuracy of 0.9911 - see the **Prediction** section below.

## Prediction <a class="anchor" id="prediction"></a>

In [12]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences

def final_predictions(x, y, x_tk, y_tk):
    """Get predictions using the final model.
    
    Parameters:
    x: Preprocessed English data
    y: Preprocessed French data
    x_tk: English tokenizer
    y_tk: French tokenizer
    """
    # Train a neural network using model_final
    model = model_final(
        x.shape,
        y.shape[1],
        len(x_tk.word_index)+1,
        len(y_tk.word_index)+1)
    print (model.summary())
    model.fit(x, y, batch_size=1024, epochs=30, validation_split=0.2)
    
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    # Print out two samples of predictions and labels
    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[x] for x in y[0].flatten()]))
    return model, predictions


final_model, predictions = final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 15)                0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 15, 300)           60000     
_________________________________________________________________
bidirectional_7 (Bidirection (None, 1024)              3330048   
_________________________________________________________________
dense_7 (Dense)              (None, 512)               524800    
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 21, 512)           0         
_________________________________________________________________
bidirectional_8 (Bidirection (None, 21, 1024)          4198400   
_________________________________________________________________
time_distributed_4 (TimeDist (None, 21, 345)           353625    
__________

In [13]:
final_rnn_model_scores = final_model.evaluate(preproc_english_sentences, preproc_french_sentences, verbose=0)
print("Model Accuracy: %.2f%%" % (final_rnn_model_scores[1]*100))

Model Accuracy: 99.11%
